In [27]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

Reading data

In [28]:
data = pd.read_csv('./ds-boot-2.csv', sep='\t')
data.head()

,id,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p24,p25,p26,p27,p28,y1,y2,y3,y4,y5
0,11,6.48148,3.0,5.0,7.75000,0.0,7.16667,8.16667,9.66667,6.16667,...,1.66667,3.16667,0.0,0.00000,0.0,1.0,5.00,7.44,1.18,4.38
1,12,5.74074,4.0,8.0,7.33333,8.0,8.83333,9.75000,9.66667,9.00000,...,2.50000,5.50000,5.0,8.66667,8.0,4.5,4.25,8.93,2.00,6.03
2,25,7.59259,7.0,8.0,7.66667,8.0,9.66667,9.50000,6.16667,9.66667,...,3.50000,3.50000,9.0,6.50000,7.0,7.5,11.00,8.97,2.00,9.12
3,31,5.96297,4.0,8.0,9.33333,10.0,9.33333,7.00000,8.50000,9.66667,...,0.00000,0.00000,0.0,0.00000,0.0,4.0,6.25,8.93,1.82,6.41
4,48,5.44444,1.0,3.5,6.41667,9.0,8.50000,7.08333,6.33333,9.00000,...,0.00000,0.00000,0.0,0.00000,0.0,0.0,3.00,8.08,1.36,3.67


In [29]:
X = data.values[:,1:29]
Y = data.values[:,29:]

### Part 1

In [104]:
def estimates(y, B=1000):
    bs_samples = np.random.choice(y,size=(B,y.shape[0]))
    bs_stat = np.mean(bs_samples,axis=1)
    est = np.mean(bs_stat)
    se = np.std(bs_stat)
    print('Mean: %f; SE=%f' % (est,se))
    bs_stat = np.median(bs_samples,axis=1)
    est = np.mean(bs_stat)
    se = np.std(bs_stat)
    print('Median: %f; SE=%f' % (est,se))

In [105]:
for i,y in enumerate(Y.T):
    print('y'+str(i+1))
    estimates(y)

y1
Mean: 4.042112; SE=0.432418
Median: 4.045250; SE=0.538298
y2
Mean: 5.510750; SE=0.546246
Median: 6.180250; SE=0.684432
y3
Mean: 7.292705; SE=0.448270
Median: 8.301710; SE=0.278300
y4
Mean: 1.532596; SE=0.106651
Median: 1.866640; SE=0.130698
y5
Mean: 5.627429; SE=0.432911
Median: 6.161550; SE=0.433187


### Part 2

In [37]:
def get_lr_coefs(X,y):
    cls = LinearRegression()
    cls.fit(X,y)
    return np.concatenate([cls.coef_, [cls.intercept_]])

In [83]:
def bootstrap_conf_int(X, Y, y_ind, B=1000):
    original_coefs = get_lr_coefs(X,Y[:,y_ind])
    bootstrap_coefs = []
    for _ in range(B):
        inds = np.random.choice(range(X.shape[0]),size=X.shape[0])
        bootstrap_coefs.append(get_lr_coefs(X[inds], Y[inds,y_ind]))
    bootstrap_coefs = np.array(bootstrap_coefs)
    deltas = original_coefs - bootstrap_coefs
    deltas_l = np.percentile(deltas, 2.5,axis = 0)
    deltas_r = np.percentile(deltas, 97.5,axis = 0)
    l_bound = original_coefs - deltas_r
    r_bound = original_coefs - deltas_l
    return pd.DataFrame([{'est': original_coefs[i],
     'l': l_bound[i],
     'r': r_bound[i]} for i in range(X.shape[1])])

Last coefficient is intercept

In [86]:
for i in range(5):
    print('y'+str(i+1))
    print(bootstrap_conf_int(X,Y, i))

y1
         est         l         r
0   0.352388 -0.417834  1.624971
1  -0.087259 -0.523897  0.489254
2  -0.472046 -0.946951  0.490285
3   1.121283 -0.227561  1.456374
4  -0.325610 -0.953048  0.390531
5   0.916609 -0.518341  1.232576
6  -1.070163 -1.661311  0.276723
7  -0.210380 -0.800414  0.620446
8  -0.184591 -0.778014  0.829778
9  -0.342690 -0.864844  0.425619
10  0.764015 -0.302638  1.197802
11  0.040134 -0.753658  0.799725
12  0.219712 -0.569503  0.914321
13 -0.132371 -0.695987  0.572967
14 -0.801004 -1.100484  0.153536
15  0.255596 -0.221054  0.679925
16  0.577988 -0.202438  0.911390
17 -0.121128 -0.653254  0.399146
18 -0.418202 -0.861865  0.403991
19 -0.259606 -0.712410  0.460751
20  0.793505 -0.187492  0.947581
21 -0.113371 -0.525927  0.449656
22  0.410392 -0.084500  0.786637
23 -0.382840 -0.792261  0.245933
24  0.603136 -0.332867  0.900529
25  0.136971 -0.510768  0.647168
26 -0.397624 -0.686877  0.502118
27 -0.103562 -0.575674  0.358694
y2
         est         l         r
0  -